In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import shutil
import sys
import numpy as np
import pandas as pd
import tensorflow as tf

/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
_CSV_COLUMNS = [
    '', 'msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 
    'target', 'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist',
    'language', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'
]

_CSV_COLUMN_DEFAULTS = [[0], ['missing_msno'], ['missing_song_id'], ['missing_tab'], ['missing_name'], ['missing_type'], 
                        [''], [241812.0], ['missing_genre_ids'], ['missing_artist_name'], ['missing_composer'], ['missing_lyricist'], 
                        [-1.0], [0], [21], ['missing_gender'], [0], [0], [0]]

parser = argparse.ArgumentParser()

parser.add_argument(
    '--model_dir', type=str, default='/tmp/census_model',
    help='Base directory for the model.')

parser.add_argument(
    '--model_type', type=str, default='deep',#'wide_deep',
    help="Valid model types: {'wide', 'deep', 'wide_deep'}.")

parser.add_argument(
    '--train_epochs', type=int, default=40, help='Number of training epochs.')

parser.add_argument(
    '--epochs_per_eval', type=int, default=2,
    help='The number of training epochs to run between evaluations.')

parser.add_argument(
    '--batch_size', type=int, default=40, help='Number of examples per batch.')

parser.add_argument(
    '--train_data', type=str, default='./labeled_train_X2.csv',
    help='Path to the training data.')

parser.add_argument(
    '--test_data', type=str, default='./labeled_train_X3.csv',
    help='Path to the test data.')

_StoreAction(option_strings=['--test_data'], dest='test_data', nargs=None, const=None, default='./labeled_train_X3.csv', type=<class 'str'>, choices=None, help='Path to the test data.', metavar=None)

In [3]:
_NUM_EXAMPLES = {
    'train': 5901935 + 1474484,
    'validation': 1000,
}


In [4]:
def build_model_columns():
  """Builds a set of wide and deep feature columns."""
  # Continuous columns
  bd = tf.feature_column.numeric_column('bd')
  song_length = tf.feature_column.numeric_column('song_length')
  city = tf.feature_column.numeric_column('city')
  language = tf.feature_column.numeric_column('language')
  registered_via = tf.feature_column.numeric_column('registered_via')
  genre_ids = tf.feature_column.categorical_column_with_hash_bucket(
      'genre_ids', hash_bucket_size=10000)

  gender = tf.feature_column.categorical_column_with_vocabulary_list(
      'gender', ['male', 'female'])

  msno = tf.feature_column.categorical_column_with_hash_bucket(
      'msno', hash_bucket_size=3200000)
  song_id = tf.feature_column.categorical_column_with_hash_bucket(
      'song_id', hash_bucket_size=2000000)
  artist_name = tf.feature_column.categorical_column_with_hash_bucket(
      'artist_name', hash_bucket_size=100000)
  composer = tf.feature_column.categorical_column_with_hash_bucket(
      'composer', hash_bucket_size=100000)
  lyricist = tf.feature_column.categorical_column_with_hash_bucket(
      'lyricist', hash_bucket_size=100000)
  # To show an example of hashing:
  source_system_tab = tf.feature_column.categorical_column_with_hash_bucket(
      'source_system_tab', hash_bucket_size=20)
  source_screen_name = tf.feature_column.categorical_column_with_hash_bucket(
      'source_screen_name', hash_bucket_size=20)
  source_type = tf.feature_column.categorical_column_with_hash_bucket(
      'source_type', hash_bucket_size=20)

  # Transformations.
  age_buckets = tf.feature_column.bucketized_column(
      bd, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
  song_length_buckets = tf.feature_column.bucketized_column(
      song_length, boundaries=[160000, 320000, 480000, 640000])

  # Wide columns and deep columns.
  base_columns = [
      gender, artist_name, source_system_tab, source_screen_name,
      source_type, composer, lyricist, age_buckets, song_id, msno,
  ]

  crossed_columns = [
      tf.feature_column.crossed_column(
          ['artist_name', 'composer', 'lyricist'], hash_bucket_size=10000),
  ]

  wide_columns = base_columns + crossed_columns

  deep_columns = [
      bd,
      city,
      language,
      tf.feature_column.numeric_column('song_length', normalizer_fn=lambda x: (x - 241812.0) / 67351),
      registered_via,
      tf.feature_column.embedding_column(genre_ids, 9),
      tf.feature_column.embedding_column(artist_name, 12),
      tf.feature_column.embedding_column(composer, 12),
      tf.feature_column.embedding_column(lyricist, 12),
      tf.feature_column.embedding_column(source_system_tab, 9),
      tf.feature_column.embedding_column(source_screen_name, 9),
      tf.feature_column.embedding_column(source_type, 9),
  ]

  return wide_columns, deep_columns

In [5]:
def build_estimator(model_dir, model_type):
  """Build an estimator appropriate for the given model type."""
  wide_columns, deep_columns = build_model_columns()
  hidden_units = [128, 128, 64, 64, 32]#[100, 75, 50, 25]

  # Create a tf.estimator.RunConfig to ensure the model is run on CPU, which
  # trains faster than GPU for this model.
  run_config = tf.estimator.RunConfig().replace(
      session_config=tf.ConfigProto(device_count={'GPU': 0}))

  if model_type == 'wide':
    return tf.estimator.LinearClassifier(
        model_dir=model_dir,
        feature_columns=wide_columns,
        config=run_config)
  elif model_type == 'deep':
    return tf.estimator.DNNClassifier(
        model_dir=model_dir,
        feature_columns=deep_columns,
        hidden_units=hidden_units,
        config=run_config)
  else:
    return tf.estimator.DNNLinearCombinedClassifier(
        model_dir=model_dir,
        linear_feature_columns=wide_columns,
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=hidden_units,
        config=run_config)

In [6]:
def input_fn(data_file, num_epochs, shuffle, batch_size):
  """Generate an input function for the Estimator."""
  assert tf.gfile.Exists(data_file), (
      '%s not found. Please make sure you have either run data_download.py or '
      'set both arguments --train_data and --test_data.' % data_file)

  def parse_csv(value):
    print('Parsing', data_file)
    columns = tf.decode_csv(value, field_delim = ',', record_defaults=_CSV_COLUMN_DEFAULTS)
    features = dict(zip(_CSV_COLUMNS, columns))
    labels = features.pop('target')
    return features, tf.equal(labels, '1')

  # Extract lines from input files using the Dataset API.
  dataset = tf.data.TextLineDataset(data_file)
  
  if shuffle:
    dataset = dataset.shuffle(buffer_size=_NUM_EXAMPLES['train'])

  dataset = dataset.map(parse_csv, num_parallel_calls=5) #skip(7376419)

  # We call repeat after shuffling, rather than before, to prevent separate
  # epochs from blending together.
  dataset = dataset.repeat(num_epochs)
  dataset = dataset.batch(batch_size)

  iterator = dataset.make_one_shot_iterator()
  features, labels = iterator.get_next()
  return features, labels

In [8]:
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

In [20]:
print("what's up?")

In [ ]:
temp = 0
temp2 = 0
model = None
def main(unused_argv):
  global temp
  global temp2
  global model
  # Clean up the model directory if present
  shutil.rmtree(FLAGS.model_dir, ignore_errors=True)
  model = build_estimator(FLAGS.model_dir, FLAGS.model_type)

  # Train and evaluate the model every `FLAGS.epochs_per_eval` epochs.
  for n in range(5):#range(FLAGS.train_epochs // FLAGS.epochs_per_eval):
    model.train(input_fn=lambda: input_fn(
        FLAGS.train_data, FLAGS.epochs_per_eval, True, FLAGS.batch_size))

    results = model.evaluate(input_fn=lambda: input_fn(
        FLAGS.test_data, 1, False, FLAGS.batch_size))

    # Display evaluation metrics
    print('Results at epoch', (n + 1) * FLAGS.epochs_per_eval)
    print('-' * 60)

    #for key in sorted(results):
    #  print('%s: %s' % (key, results[key]))
  #model.export_savedmodel(export_dir_base='./')
  column_names2 = [
    'id', 'msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type',
    'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist',
    'language', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'
  ] 
  df = pd.read_csv('./labeled_test_X2.csv', index_col=False, names=column_names2,
                            skip_blank_lines=True, keep_default_na=False)#, dtype={'city':str, 'language':str})
  predictions = list(model.predict(input_fn=tf.estimator.inputs.pandas_input_fn(
      x=df, num_epochs=1,shuffle=False)))
  res = pd.DataFrame(predictions)
  temp = model
  temp2 = predictions
  res.to_csv('pred.csv', sep = ',', header = False)
  

if __name__ == '__main__':
  tf.logging.set_verbosity(tf.logging.INFO)
  FLAGS, unparsed = parser.parse_known_args()
  tf.app.run(main=main, argv=[sys.argv[0]] + unparsed + [temp])

INFO:tensorflow:Using config: {'_model_dir': '/tmp/census_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': device_count {
  key: "GPU"
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f0911f36da0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Parsing ./labeled_train_X2.csv
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/census_model/model.ckpt.
INFO:tensorflow:loss = 29.7325, step = 1
INFO:tensorflow:global_step/sec: 114.49
INFO:tensorflow:loss = 26.0883, step = 101 (0.875 sec)
INFO:tensorflow:global_step/sec: 198.507
INFO:tensorflow:loss = 27.2159, step = 201 (0.504 sec)
INFO:tensorflow:global_step/sec: 196.633
INFO:ten

INFO:tensorflow:global_step/sec: 212.431
INFO:tensorflow:loss = 26.3604, step = 7801 (0.471 sec)
INFO:tensorflow:global_step/sec: 212.75
INFO:tensorflow:loss = 24.148, step = 7901 (0.470 sec)
INFO:tensorflow:global_step/sec: 211.997
INFO:tensorflow:loss = 22.3574, step = 8001 (0.472 sec)
INFO:tensorflow:global_step/sec: 211.696
INFO:tensorflow:loss = 25.1339, step = 8101 (0.472 sec)
INFO:tensorflow:global_step/sec: 214.944
INFO:tensorflow:loss = 27.2721, step = 8201 (0.465 sec)
INFO:tensorflow:global_step/sec: 213.808
INFO:tensorflow:loss = 26.8422, step = 8301 (0.468 sec)
INFO:tensorflow:global_step/sec: 212.392
INFO:tensorflow:loss = 26.351, step = 8401 (0.471 sec)
INFO:tensorflow:global_step/sec: 212.566
INFO:tensorflow:loss = 24.9972, step = 8501 (0.471 sec)
INFO:tensorflow:global_step/sec: 213.054
INFO:tensorflow:loss = 23.8465, step = 8601 (0.469 sec)
INFO:tensorflow:global_step/sec: 213.806
INFO:tensorflow:loss = 24.0721, step = 8701 (0.468 sec)
INFO:tensorflow:global_step/sec: 

INFO:tensorflow:loss = 27.4133, step = 16201 (0.471 sec)
INFO:tensorflow:global_step/sec: 210.702
INFO:tensorflow:loss = 26.0451, step = 16301 (0.475 sec)
INFO:tensorflow:global_step/sec: 211.089
INFO:tensorflow:loss = 26.0425, step = 16401 (0.474 sec)
INFO:tensorflow:global_step/sec: 212.822
INFO:tensorflow:loss = 25.6191, step = 16501 (0.470 sec)
INFO:tensorflow:global_step/sec: 209.743
INFO:tensorflow:loss = 26.7169, step = 16601 (0.477 sec)
INFO:tensorflow:global_step/sec: 211.194
INFO:tensorflow:loss = 24.9239, step = 16701 (0.473 sec)
INFO:tensorflow:global_step/sec: 210.844
INFO:tensorflow:loss = 25.418, step = 16801 (0.474 sec)
INFO:tensorflow:global_step/sec: 209.429
INFO:tensorflow:loss = 24.9603, step = 16901 (0.478 sec)
INFO:tensorflow:global_step/sec: 213.051
INFO:tensorflow:loss = 28.1434, step = 17001 (0.469 sec)
INFO:tensorflow:global_step/sec: 210.936
INFO:tensorflow:loss = 25.7263, step = 17101 (0.474 sec)
INFO:tensorflow:global_step/sec: 212.67
INFO:tensorflow:loss =

INFO:tensorflow:loss = 26.4413, step = 24601 (0.469 sec)
INFO:tensorflow:global_step/sec: 215.205
INFO:tensorflow:loss = 23.9747, step = 24701 (0.465 sec)
INFO:tensorflow:global_step/sec: 214.641
INFO:tensorflow:loss = 22.649, step = 24801 (0.466 sec)
INFO:tensorflow:global_step/sec: 211.713
INFO:tensorflow:loss = 25.3595, step = 24901 (0.472 sec)
INFO:tensorflow:global_step/sec: 212.915
INFO:tensorflow:loss = 25.0829, step = 25001 (0.470 sec)
INFO:tensorflow:global_step/sec: 213.346
INFO:tensorflow:loss = 25.457, step = 25101 (0.469 sec)
INFO:tensorflow:global_step/sec: 216.829
INFO:tensorflow:loss = 24.2152, step = 25201 (0.461 sec)
INFO:tensorflow:global_step/sec: 214.205
INFO:tensorflow:loss = 22.6986, step = 25301 (0.467 sec)
INFO:tensorflow:global_step/sec: 214.189
INFO:tensorflow:loss = 27.1225, step = 25401 (0.467 sec)
INFO:tensorflow:global_step/sec: 213.475
INFO:tensorflow:loss = 27.6572, step = 25501 (0.469 sec)
INFO:tensorflow:global_step/sec: 212.159
INFO:tensorflow:loss =

INFO:tensorflow:loss = 23.6309, step = 33001 (0.465 sec)
INFO:tensorflow:global_step/sec: 216.93
INFO:tensorflow:loss = 24.9188, step = 33101 (0.461 sec)
INFO:tensorflow:global_step/sec: 218.385
INFO:tensorflow:loss = 26.4951, step = 33201 (0.458 sec)
INFO:tensorflow:global_step/sec: 213.572
INFO:tensorflow:loss = 27.0658, step = 33301 (0.468 sec)
INFO:tensorflow:global_step/sec: 213.779
INFO:tensorflow:loss = 23.4466, step = 33401 (0.467 sec)
INFO:tensorflow:global_step/sec: 214.948
INFO:tensorflow:loss = 25.2411, step = 33501 (0.465 sec)
INFO:tensorflow:global_step/sec: 213.469
INFO:tensorflow:loss = 23.9481, step = 33601 (0.468 sec)
INFO:tensorflow:global_step/sec: 215.273
INFO:tensorflow:loss = 25.9804, step = 33701 (0.464 sec)
INFO:tensorflow:global_step/sec: 216.531
INFO:tensorflow:loss = 26.1175, step = 33801 (0.462 sec)
INFO:tensorflow:global_step/sec: 213.825
INFO:tensorflow:loss = 27.8186, step = 33901 (0.468 sec)
INFO:tensorflow:global_step/sec: 216.243
INFO:tensorflow:loss 

...

INFO:tensorflow:loss = 24.1476, step = 174901 (0.523 sec)
INFO:tensorflow:global_step/sec: 197.865
INFO:tensorflow:loss = 25.6209, step = 175001 (0.505 sec)
INFO:tensorflow:global_step/sec: 199.677
INFO:tensorflow:loss = 26.9865, step = 175101 (0.501 sec)
INFO:tensorflow:global_step/sec: 198.643
INFO:tensorflow:loss = 24.6673, step = 175201 (0.503 sec)
INFO:tensorflow:global_step/sec: 196.347
INFO:tensorflow:loss = 22.5627, step = 175301 (0.509 sec)
INFO:tensorflow:global_step/sec: 194.144
INFO:tensorflow:loss = 23.3086, step = 175401 (0.515 sec)
INFO:tensorflow:global_step/sec: 195.719
INFO:tensorflow:loss = 26.818, step = 175501 (0.511 sec)
INFO:tensorflow:global_step/sec: 195.425
INFO:tensorflow:loss = 25.959, step = 175601 (0.512 sec)
INFO:tensorflow:global_step/sec: 193.07
INFO:tensorflow:loss = 26.8623, step = 175701 (0.518 sec)
INFO:tensorflow:global_step/sec: 194.947
INFO:tensorflow:loss = 26.6171, step = 175801 (0.513 sec)
INFO:tensorflow:global_step/sec: 194.169
INFO:tensorfl

INFO:tensorflow:loss = 28.1013, step = 183201 (0.843 sec)
INFO:tensorflow:global_step/sec: 116.219
INFO:tensorflow:loss = 22.288, step = 183301 (0.860 sec)
INFO:tensorflow:global_step/sec: 115.269
INFO:tensorflow:loss = 27.8811, step = 183401 (0.867 sec)
INFO:tensorflow:global_step/sec: 112.24
INFO:tensorflow:loss = 28.2057, step = 183501 (0.891 sec)
INFO:tensorflow:global_step/sec: 112.188
INFO:tensorflow:loss = 26.7307, step = 183601 (0.891 sec)
INFO:tensorflow:global_step/sec: 109.458
INFO:tensorflow:loss = 24.666, step = 183701 (0.914 sec)
INFO:tensorflow:global_step/sec: 109.05
INFO:tensorflow:loss = 27.4045, step = 183801 (0.917 sec)
INFO:tensorflow:global_step/sec: 106.363
INFO:tensorflow:loss = 24.4815, step = 183901 (0.940 sec)
INFO:tensorflow:global_step/sec: 108.925
INFO:tensorflow:loss = 26.1802, step = 184001 (0.918 sec)
INFO:tensorflow:global_step/sec: 110.912
INFO:tensorflow:loss = 26.2106, step = 184101 (0.902 sec)
INFO:tensorflow:global_step/sec: 113.308
INFO:tensorflo

INFO:tensorflow:loss = 22.5164, step = 191501 (0.462 sec)
INFO:tensorflow:global_step/sec: 212.176
INFO:tensorflow:loss = 25.4096, step = 191601 (0.471 sec)
INFO:tensorflow:global_step/sec: 215.255
INFO:tensorflow:loss = 21.7918, step = 191701 (0.465 sec)
INFO:tensorflow:global_step/sec: 214.041
INFO:tensorflow:loss = 26.3007, step = 191801 (0.467 sec)
INFO:tensorflow:global_step/sec: 214.427
INFO:tensorflow:loss = 25.359, step = 191901 (0.466 sec)
INFO:tensorflow:global_step/sec: 213.296
INFO:tensorflow:loss = 22.9873, step = 192001 (0.469 sec)
INFO:tensorflow:global_step/sec: 211.756
INFO:tensorflow:loss = 23.7353, step = 192101 (0.472 sec)
INFO:tensorflow:global_step/sec: 212.63
INFO:tensorflow:loss = 26.3782, step = 192201 (0.470 sec)
INFO:tensorflow:global_step/sec: 213.002
INFO:tensorflow:loss = 23.9457, step = 192301 (0.469 sec)
INFO:tensorflow:global_step/sec: 212.558
INFO:tensorflow:loss = 21.6553, step = 192401 (0.470 sec)
INFO:tensorflow:global_step/sec: 212.341
INFO:tensorf

INFO:tensorflow:loss = 28.6172, step = 199801 (0.468 sec)
INFO:tensorflow:global_step/sec: 215.828
INFO:tensorflow:loss = 22.3416, step = 199901 (0.463 sec)
INFO:tensorflow:global_step/sec: 216.932
INFO:tensorflow:loss = 24.9302, step = 200001 (0.461 sec)
INFO:tensorflow:global_step/sec: 216.007
INFO:tensorflow:loss = 24.5085, step = 200101 (0.463 sec)
INFO:tensorflow:global_step/sec: 215.4
INFO:tensorflow:loss = 25.9499, step = 200201 (0.464 sec)
INFO:tensorflow:global_step/sec: 214.146
INFO:tensorflow:loss = 29.2084, step = 200301 (0.467 sec)
INFO:tensorflow:global_step/sec: 214.672
INFO:tensorflow:loss = 21.1211, step = 200401 (0.466 sec)
INFO:tensorflow:global_step/sec: 214.659
INFO:tensorflow:loss = 24.5386, step = 200501 (0.466 sec)
INFO:tensorflow:global_step/sec: 215.045
INFO:tensorflow:loss = 26.6268, step = 200601 (0.465 sec)
INFO:tensorflow:global_step/sec: 216.3
INFO:tensorflow:loss = 22.7607, step = 200701 (0.462 sec)
INFO:tensorflow:global_step/sec: 214.955
INFO:tensorflo

INFO:tensorflow:loss = 27.6448, step = 208101 (0.467 sec)
INFO:tensorflow:global_step/sec: 216.434
INFO:tensorflow:loss = 23.9076, step = 208201 (0.462 sec)
INFO:tensorflow:global_step/sec: 215.101
INFO:tensorflow:loss = 24.8828, step = 208301 (0.465 sec)
INFO:tensorflow:global_step/sec: 217.056
INFO:tensorflow:loss = 23.231, step = 208401 (0.461 sec)
INFO:tensorflow:global_step/sec: 214.15
INFO:tensorflow:loss = 25.3952, step = 208501 (0.467 sec)
INFO:tensorflow:global_step/sec: 216.109
INFO:tensorflow:loss = 27.9911, step = 208601 (0.463 sec)
INFO:tensorflow:global_step/sec: 214.63
INFO:tensorflow:loss = 24.5859, step = 208701 (0.466 sec)
INFO:tensorflow:global_step/sec: 217.666
INFO:tensorflow:loss = 25.1205, step = 208801 (0.460 sec)
INFO:tensorflow:global_step/sec: 215.692
INFO:tensorflow:loss = 24.318, step = 208901 (0.464 sec)
INFO:tensorflow:global_step/sec: 213.554
INFO:tensorflow:loss = 25.1427, step = 209001 (0.468 sec)
INFO:tensorflow:global_step/sec: 212.984
INFO:tensorflo

In [ ]:
column_names2 = [
    'id', 'msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type',
    'song_length', 'genre_ids', 'artist_name', 'composer', 'lyricist',
    'language', 'city', 'bd', 'gender', 'registered_via', 'registration_init_time', 'expiration_date'
  ] 
df = pd.read_csv('./labeled_test_X2.csv', index_col=False, names=column_names2,
                        skip_blank_lines=True, keep_default_na=False)
predictions = list(temp.predict(input_fn=tf.estimator.inputs.pandas_input_fn(
  x=df, num_epochs=1,shuffle=False)))
res = pd.DataFrame(predictions)

In [26]:

column_names3=['id', 'target', 'score', 's1', 's2', 's']
def conv3(val):  
    return np.int(val[1])
census_pred = pd.read_csv('./pred.csv', index_col=False, names=column_names3,
                          skip_blank_lines=True, keep_default_na=False, converters={'target':conv3})

In [27]:
header=['id', 'target']
census_pred.to_csv('output.csv', index=False, columns = header)

In [40]:
np.sum(census_pred['target'])

84351